In [214]:
from analytics_toolbox_api.network.api import Api # сама либа с api
from analytics_toolbox_api.common.operations import Select # часть, которая отвечает за запросы типа Select
from analytics_toolbox_api.templates.pipeline_templates import print_queues  #часть, которая отвечает за то, чтобы можно было посмотреть, какая сейчас загрузка очередей
from analytics_toolbox_api.common.pipeline import Pipeline
from analytics_toolbox_api.common.operations import Journal
from analytics_toolbox_api.common.operations import SelectById, AggregationById, ABById

#создаем себе клиент, с помощью которого будем ходить
client = Api('v.tikhonov', '1LKY!PcLV$',env='vk_services')

In [104]:
from time import sleep
from analytics_toolbox_api.templates.pipeline_templates import print_queues
from IPython.display import clear_output


def get_df_status(select, sleep_sec=5):
    while select.status()!='finished':
        print(select.status())
        print_queues(client)
        sleep(sleep_sec)
        clear_output(wait=True)

In [4]:
sql = '''
WITH t1 AS (SELECT user_id
FROM community_actions
WHERE dt = '$dt'
AND user_role IN ('ADMIN', 'CREATOR', 'MODERATOR', 'EDITOR')
AND event = 'PUBLISH_POST'),

t2 AS (
SELECT 
    user_id, 
    attachments.type, 
    attachments.owner_id, 
    attachments.item_id
FROM
    (SELECT 
        user_id, 
        attachments.type, 
        attachments.owner_id, 
        attachments.item_id
    FROM posting
    WHERE dt = '$dt'
    AND event_type = 'post_now'
    AND owner_id < 0
    AND has(attachments.type, 'photo'))
ARRAY JOIN attachments.type, attachments.owner_id, attachments.item_id
WHERE attachments.type = 'photo'
),

t4 AS(
SELECT user_id, photo_id 
FROM upload_photo 
WHERE dt = '$dt'
GROUP BY user_id, photo_id 
)

SELECT count()
FROM t1
GLOBAL ANY LEFT JOIN t2 USING user_id
'''

In [109]:
### кол-во фото из фото сообществ

In [5]:
big_sql = '''
WITH t1 AS(
SELECT 
        user_id, 
        attachments.type, 
        attachments.owner_id, 
        attachments.item_id
    FROM
        (SELECT 
            user_id,
            attachments.type, 
            attachments.owner_id, 
            attachments.item_id
        FROM posting
        WHERE dt = yesterday()
        AND event_type = 'post_now'
        AND owner_id < 0
        AND has(attachments.type, 'photo'))
    ARRAY JOIN attachments.type, attachments.owner_id, attachments.item_id
    WHERE attachments.type = 'photo'
    and intHash64(attachments.owner_id) % $shards_num = $shard
),

t2 AS (
SELECT owner_id, photo_id 
FROM upload_photo 
WHERE dt = '$dt'
and intHash64(owner_id) % $shards_num = $shard
GROUP BY owner_id, photo_id 
)

SELECT user_id
FROM t1
GLOBAL ANY LEFT JOIN t2 ON t1.attachments.owner_id = t2.owner_id AND t1.attachments.item_id = t2.photo_id
'''

In [19]:
UP_select = Select(client, big_sql, granularity=1, dates = '2024-06-11 2024-06-25')

In [20]:
UP_select.run()

{'operation_id': '03493f6d-d94d-4076-b606-b3d228bbd928',
 'agg_operation_id': '19548fe7-4379-4da7-b118-1c339bbff99a',
 'replica_num': 1,
 'host': 'ch814601',
 'table_name': 'vtikhonov_2024_06_26_18_44_40_749392',
 'queue_position': 1,
 'pipeline_id': '19548fe7-4379-4da7-b118-1c339bbff99a',
 'check_version': False}

In [7]:
UP_select = SelectById(client, 'd21db880-d304-472a-a1d5-83056c8f650b')

In [8]:
UP_select.status()

'finished'

In [10]:
CA_sql = '''
SELECT user_id
FROM community_actions
WHERE dt = yesterday()
AND user_role IN ('ADMIN', 'CREATOR', 'MODERATOR', 'EDITOR')
AND event = 'PUBLISH_POST'
'''

In [13]:
CA_select = Select(client, sql=CA_sql)
CA_select.run()

{'operation_id': 'e5479d5d-a93f-4c28-8934-4f269d88c95e',
 'agg_operation_id': 'cdc7151b-1f8b-4000-987f-4a39bceeadf2',
 'replica_num': 0,
 'host': 'ch814600',
 'table_name': 'vtikhonov_2024_06_26_22_07_15_872763',
 'queue_position': 0,
 'pipeline_id': 'cdc7151b-1f8b-4000-987f-4a39bceeadf2',
 'check_version': False}

In [80]:
CA_select = SelectById(client, 'e5479d5d-a93f-4c28-8934-4f269d88c95e')

In [21]:
CA_select.status()

'finished'

In [54]:
j = Journal(client, 'select', operation_id_list=['d21db880-d304-472a-a1d5-83056c8f650b', 'e5479d5d-a93f-4c28-8934-4f269d88c95e']).run()
print(j[0]['cluster'], j[0]['insert_table'])

product_analytics_3 vtikhonov_2024_06_26_22_07_15_872763


In [55]:
print(j[1]['cluster'], j[1]['insert_table'])

product_analytics_3 vtikhonov_2024_06_26_17_56_20_455634


In [83]:
res1_sql = '''
SELECT user_id
FROM temp.vtikhonov_2024_06_26_17_56_20_455634
GROUP BY user_id
'''

In [84]:
res_select1 = Select(client, res1_sql, cluster='product_analytics_test')
res_select1.run()

{'operation_id': '9032dc06-d031-41ad-911c-4cd4a80d1391',
 'agg_operation_id': '511e07c1-fad2-4817-af58-a868b6474f94',
 'replica_num': 1,
 'host': 'ch814601',
 'table_name': 'vtikhonov_2024_06_26_23_00_36_962410',
 'queue_position': 0,
 'pipeline_id': '511e07c1-fad2-4817-af58-a868b6474f94',
 'check_version': False}

In [86]:
get_df_status(res_select1)

running agg 0/1
04_short
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  23:00:43  |     -      |   2700  |  0  |  0   |   t.isakova   |d724d5b9-41bf-4aae-808d-ac010a078b00|

06_long_other_cluster
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  19:05:49  |  19:05:56  |  21000  |  1  |  0   |e.asmandiyarova|da32ccf3-cfda-4c56-994e-b2720fd77b85|
 select |  19:51:30  |  19:51:52  |  21000  |  1  |  0   |e.asmandiyarova|9df54a58-c13a-4fcc-b4c6-3e91eeb74887|

07_other_cluster
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
--------------------------------------

In [87]:
res2_sql = '''
SELECT user_id, count() AS count
FROM temp.vtikhonov_2024_06_26_22_07_15_872763
GROUP BY user_id
'''

In [91]:
res_select2 = Select(client, res2_sql, cluster='product_analytics_test')
res_select2.run()

{'operation_id': '9ea34524-b2b0-4436-b041-aefc75c8f26d',
 'agg_operation_id': '044b58a7-7850-45c8-86c9-4e4984b771c6',
 'replica_num': 0,
 'host': 'ch814600',
 'table_name': 'vtikhonov_2024_06_26_23_03_35_976155',
 'queue_position': 0,
 'pipeline_id': '044b58a7-7850-45c8-86c9-4e4984b771c6',
 'check_version': False}

In [93]:
res_select2.status()

'finished'

In [94]:
j1 = Journal(client, 'select', operation_id_list=['9032dc06-d031-41ad-911c-4cd4a80d1391', '9ea34524-b2b0-4436-b041-aefc75c8f26d']).run()
print(j1[0]['cluster'], j1[0]['insert_table'])

product_analytics_test vtikhonov_2024_06_26_23_03_35_976155


In [95]:
print(j1[1]['cluster'], j1[1]['insert_table'])

product_analytics_test vtikhonov_2024_06_26_23_00_36_962410


In [100]:
res_sql = '''
SELECT sum(count)
FROM temp.vtikhonov_2024_06_26_23_03_35_976155 t1
INNER JOIN temp.vtikhonov_2024_06_26_23_00_36_962410 t2 USING user_id
'''

In [101]:
res = Select(client, res_sql, cluster='product_analytics_test')
res.run()

{'operation_id': '70a8ec44-256a-444f-8495-27d881465a32',
 'agg_operation_id': 'f112478f-ec54-4520-b3cc-4e83acb3b444',
 'replica_num': 1,
 'host': 'ch814601',
 'table_name': 'vtikhonov_2024_06_26_23_08_39_451798',
 'queue_position': 0,
 'pipeline_id': 'f112478f-ec54-4520-b3cc-4e83acb3b444',
 'check_version': False}

In [102]:
get_df_status(res)

running agg 0/1
07_other_cluster
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  23:08:39  |  23:08:40  |   900   |  1  |  0   |   v.tikhonov  |70a8ec44-256a-444f-8495-27d881465a32|

10_etl_long
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
  agg   |  21:29:35  |     -      |  12100  |  0  |  0   | team_marketing|6e10dbf7-7e7d-432f-917d-49e8912f9c61|
  agg   |  21:29:35  |     -      |  12100  |  0  |  0   | team_marketing|8a6aa490-3351-4df8-bc60-3ab0055268b9|
  agg   |  21:29:35  |     -      |  12100  |  0  |  0   | team_marketing|65e592ce-9df3-4c6a-9fef-bd2576b2b656|
  agg   |  21:29:35  |     -      |  12100  |  0  |  0   |

In [103]:
res.get_df(wait=True)

,sum(count)
0,1964792


In [108]:
### кол-во постов в день

In [81]:
posts = CA_select.get_df()
posts.count()

user_id    3160668
dtype: int64

In [110]:
### кол-во прикрепленных фото в день

In [112]:
photo_posting_sql = '''
SELECT
    user_id,
    count() AS count
FROM
    (SELECT 
        user_id, 
        attachments.type, 
        attachments.owner_id, 
        attachments.item_id
    FROM
        (SELECT 
            user_id,
            attachments.type, 
            attachments.owner_id, 
            attachments.item_id
        FROM posting
        WHERE dt = yesterday()
        AND event_type = 'post_now'
        AND has(attachments.type, 'photo'))
    ARRAY JOIN attachments.type, attachments.owner_id, attachments.item_id
    WHERE attachments.type = 'photo'
    and intHash64(attachments.owner_id) % $shards_num = $shard)
GROUP BY user_id
'''

In [114]:
photo_posting_select = Select(client, photo_posting_sql, granularity=1)
photo_posting_select.run()

{'operation_id': '28836bb8-91cc-4647-85ab-779594f319e6',
 'agg_operation_id': '01e3d9cc-4897-4304-b056-68c88052860d',
 'replica_num': 0,
 'host': 'ch814600',
 'table_name': 'vtikhonov_2024_06_26_23_19_44_189214',
 'queue_position': 1,
 'pipeline_id': '01e3d9cc-4897-4304-b056-68c88052860d',
 'check_version': False}

In [118]:
get_df_status(photo_posting_select)


In [119]:
j2 = Journal(client, 'select', operation_id_list=['28836bb8-91cc-4647-85ab-779594f319e6']).run()
j2[0]['insert_table']

'vtikhonov_2024_06_26_23_19_44_189214'

In [120]:
photos_res_sql = '''
SELECT sum(count)
FROM temp.vtikhonov_2024_06_26_23_03_35_976155 
INNER JOIN temp.vtikhonov_2024_06_26_23_19_44_189214 USING user_id
'''

In [121]:
res2 = Select(client, photos_res_sql, cluster='product_analytics_test')

In [122]:
res2.run()

{'operation_id': '8208b236-5ee5-4b09-bf7d-c0ddd10d14d8',
 'agg_operation_id': 'b0b71709-8e0f-47f9-94da-7c41735480eb',
 'replica_num': 0,
 'host': 'ch814600',
 'table_name': 'vtikhonov_2024_06_26_23_24_47_486790',
 'queue_position': 0,
 'pipeline_id': 'b0b71709-8e0f-47f9-94da-7c41735480eb',
 'check_version': False}

In [123]:
get_df_status(res2)

running agg 0/1
03_normal
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  23:23:34  |  23:23:34  |   2700  |  31 |  6   | e.kozhevnikov |4081018f-4736-481d-9111-1ef711c757d6|
 select |  23:24:08  |  23:24:10  |   900   |  1  |  0   |alena.arsenteva|02d59f56-5983-4914-97b4-edbf4374f108|

07_other_cluster
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  23:24:47  |  23:24:48  |   900   |  1  |  0   |   v.tikhonov  |8208b236-5ee5-4b09-bf7d-c0ddd10d14d8|

10_etl_long
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
-----------------------------------------------

In [125]:
res2.get_df()

,sum(count)
0,11611651


In [126]:
###  кол-во прикрепоенных фото из фото профиля

In [145]:
profile_photo_sql = '''
SELECT
    user_id,
    count() AS count
FROM
    (SELECT 
        user_id, 
        attachments.type, 
        attachments.owner_id, 
        attachments.item_id
    FROM
        (SELECT 
            user_id,
            attachments.type, 
            attachments.owner_id, 
            attachments.item_id
        FROM posting
        WHERE dt = yesterday()
        AND event_type = 'post_now'
        AND owner_id > 0
        AND has(attachments.type, 'photo'))
    ARRAY JOIN attachments.type, attachments.owner_id, attachments.item_id
    WHERE attachments.type = 'photo'
    and intHash64(attachments.owner_id) % $shards_num = $shard)
GROUP BY user_id
'''

In [146]:
profile_posting_select = Select(client, profile_photo_sql, granularity=1)
profile_posting_select.run()

{'operation_id': '03ce1714-9551-42b6-8f3b-587ac4dfcb7f',
 'agg_operation_id': '4c7516d5-94b8-442e-aeaf-89c7fb33f2de',
 'replica_num': 1,
 'host': 'ch814601',
 'table_name': 'vtikhonov_2024_06_26_23_36_00_137784',
 'queue_position': 0,
 'pipeline_id': '4c7516d5-94b8-442e-aeaf-89c7fb33f2de',
 'check_version': False}

In [147]:
get_df_status(profile_posting_select)

running agg 0/1
02_long
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  23:35:49  |  23:35:50  |  21600  |  1  |  0   | e.kozhevnikov |ab4eaeb4-41e1-42a9-98f3-e2c59675f4b7|

03_normal
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  23:36:00  |  23:36:01  |   2700  |  1  |  0   |   v.tikhonov  |03ce1714-9551-42b6-8f3b-587ac4dfcb7f|

10_etl_long
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
  agg   |  21:29:35  |     -      |  12100  |  0  |  0  

In [148]:
j2 = Journal(client, 'select', operation_id_list=['03ce1714-9551-42b6-8f3b-587ac4dfcb7f']).run()
j2[0]['insert_table']

'vtikhonov_2024_06_26_23_36_00_137784'

In [149]:
photosp_res_sql = '''
SELECT sum(count)
FROM temp.vtikhonov_2024_06_26_23_03_35_976155 
INNER JOIN temp.vtikhonov_2024_06_26_23_36_00_137784 USING user_id
'''

In [150]:
res3 = Select(client, photosp_res_sql, cluster='product_analytics_test')
res3.run()

{'operation_id': 'c8c4fe22-bd21-4e55-b061-b055b49a5230',
 'agg_operation_id': '2abf4121-1e5c-4cae-a518-500a97cee131',
 'replica_num': 0,
 'host': 'ch814600',
 'table_name': 'vtikhonov_2024_06_26_23_36_41_915347',
 'queue_position': 0,
 'pipeline_id': '2abf4121-1e5c-4cae-a518-500a97cee131',
 'check_version': False}

In [151]:
get_df_status(res3)

running agg 0/1
03_normal
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  23:36:54  |  23:36:55  |   900   |  1  |  0   |   m.bavykina  |7cb9421b-7222-4731-851e-3a252eefc5c6|

07_other_cluster
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  23:36:41  |  23:36:42  |   900   |  1  |  0   |   v.tikhonov  |c8c4fe22-bd21-4e55-b061-b055b49a5230|

10_etl_long
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
  agg   |  21:29:35  |     -      |  12100  |  

In [152]:
res3.get_df()

,sum(count)
0,383843


In [153]:
### кол-во прикрепленных видео в день

In [154]:
video_posting_sql = '''
SELECT
    user_id,
    count() AS count
FROM
    (SELECT 
        user_id, 
        attachments.type, 
        attachments.owner_id, 
        attachments.item_id
    FROM
        (SELECT 
            user_id,
            attachments.type, 
            attachments.owner_id, 
            attachments.item_id
        FROM posting
        WHERE dt = yesterday()
        AND event_type = 'post_now'
        AND has(attachments.type, 'video'))
    ARRAY JOIN attachments.type, attachments.owner_id, attachments.item_id
    WHERE attachments.type = 'video'
    and intHash64(attachments.owner_id) % $shards_num = $shard)
GROUP BY user_id
'''

In [155]:
video_posting_select = Select(client, video_posting_sql, granularity=1)
video_posting_select.run()

{'operation_id': 'c297c490-dbe9-451b-b326-134b0516e7c4',
 'agg_operation_id': 'dc902b68-2bd3-4e9d-adf8-8bc17bdd092a',
 'replica_num': 1,
 'host': 'ch814601',
 'table_name': 'vtikhonov_2024_06_26_23_50_20_137063',
 'queue_position': 0,
 'pipeline_id': 'dc902b68-2bd3-4e9d-adf8-8bc17bdd092a',
 'check_version': False}

In [156]:
video_posting_select.status()

'finished'

In [157]:
j3 = Journal(client, 'select', operation_id_list=['c297c490-dbe9-451b-b326-134b0516e7c4']).run()
j3[0]['insert_table']

'vtikhonov_2024_06_26_23_50_20_137063'

In [158]:
video_res_sql = '''
SELECT sum(count)
FROM temp.vtikhonov_2024_06_26_23_03_35_976155 
INNER JOIN temp.vtikhonov_2024_06_26_23_50_20_137063 USING user_id
'''

In [159]:
res3 = Select(client, video_res_sql, cluster='product_analytics_test')

In [160]:
res3.run()

{'operation_id': '487aeb83-110b-493c-91cc-5de03854382e',
 'agg_operation_id': '536316e0-0b2e-44d9-98ba-2cf06a7b794c',
 'replica_num': 0,
 'host': 'ch814600',
 'table_name': 'vtikhonov_2024_06_26_23_54_46_17526',
 'queue_position': 0,
 'pipeline_id': '536316e0-0b2e-44d9-98ba-2cf06a7b794c',
 'check_version': False}

In [162]:
res3.status()

'finished'

In [163]:
res3.get_df()

,sum(count)
0,2725472


In [243]:
### кол-во прикрепленных видео сообщества в день

In [262]:
sql = '''
WITH t1 AS (
SELECT
    user_id,
    event_id,
    video_id, 
    video_owner_id
FROM video_publish
WHERE dt >= yesterday() - 14
AND event_type = 'publish_video'
and intHash64(user_id) % $shards_num = $shard
GROUP BY user_id, event_id, video_id, video_owner_id
),

t2 AS (
SELECT 
    user_id, 
    attachments.type, 
    attachments.owner_id, 
    attachments.item_id
FROM
    (SELECT 
        user_id,
        event_id,
        attachments.type, 
        attachments.owner_id, 
        attachments.item_id
    FROM posting 
    WHERE dt = yesterday()
    AND event_type LIKE 'post_%'
    AND owner_id < 0
    AND has(attachments.type, 'video')
    AND         intHash64(user_id) % $shards_num = $shard
    GROUP BY user_id, event_id, attachments.type, attachments.owner_id, attachments.item_id)
ARRAY JOIN attachments.type, attachments.owner_id, attachments.item_id
WHERE attachments.type = 'video'
),

t3 AS (
SELECT *
FROM
    (SELECT 
        user_id,
        owner_id,
        event_id,
        attachments.type, 
        attachments.owner_id, 
        attachments.item_id
    FROM posting 
    WHERE dt = yesterday()
    AND event_type LIKE 'post_%'
    AND has(attachments.type, 'video')
    AND intHash64(user_id) % $shards_num = $shard)
ARRAY JOIN attachments.owner_id, attachments.item_id, attachments.type
WHERE attachments.type = 'video'
AND (attachments.owner_id = user_id OR (attachments.owner_id != user_id AND attachments.owner_id != owner_id))
)
    
SELECT *
FROM t3
WHERE concat(toString(attachments.item_id), toString(attachments.owner_id)) NOT IN (
    SELECT
        concat(toString(video_id), toString(video_owner_id))
    FROM t2
    INNER JOIN t1 ON t1.video_id = t2.attachments.item_id AND t1.video_owner_id = t2.attachments.owner_id
     )
'''

In [263]:
lol = Select(client, sql, granularity=1)
lol.run()

{'operation_id': '9f782174-c6ec-4042-94d9-eca55e42465e',
 'agg_operation_id': 'cceca284-0792-4bdc-bc43-af89b67f2147',
 'replica_num': 0,
 'host': 'ch814600',
 'table_name': 'vtikhonov_2024_06_28_20_02_09_311629',
 'queue_position': 0,
 'pipeline_id': 'cceca284-0792-4bdc-bc43-af89b67f2147',
 'check_version': False}

In [264]:
get_df_status(lol)

running agg 0/1
02_long
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
  agg   |  18:26:06  |  18:26:06  |  21600  | 226 |  55  |    o.belov    |be5ee9a6-7705-4804-9881-faad3d8efb0d|

03_normal
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  00:17:41  |     -      |   900   |  0  |  0   |   v.repnikov  |c13398d3-455a-4f7f-b107-97c313507e26|
 select |  00:17:41  |     -      |   900   |  0  |  0   |   v.repnikov  |2f914087-6fdf-4e52-9605-e61d9ebb8bf5|
 select |  00:17:41  |     -      |   900   |  0  |  0   |   v.repnikov  |499ae460-b379-4507-bdb5-531f738d1798|
 select |  00:17:42  |     -      |   900   |  0  |  0   |   v.repnik

In [266]:
kek = lol.get_df()
kek

In [269]:
kek

,user_id,owner_id,event_id,attachments.type,attachments.owner_id,attachments.item_id
0,63680825,63680825,1673260609,video,63680825,456239728
1,63680825,63680825,933094186,video,63680825,456239731
2,554467967,554467967,364053285,video,554467967,456239760
3,605248655,605248655,1163497488,video,605248655,456239096
4,714869439,714869439,338622156,video,-174767238,456239497
...,...,...,...,...,...,...
386870,777552865,777552865,816500529,video,777552865,456239074
386871,156727679,156727679,1748435957,video,156727679,456239530
386872,850757601,850757601,1739262755,video,850757601,456239025
386873,589806870,589806870,1075967202,video,589806870,456240621


#### кол-во прикрепленных видео в день из сообщества

In [242]:
208672 + 136825

345497

In [165]:
### Видео из сообщества (добавил в сообещство свое видео) + видео из ютубов и тд

In [238]:
sql_top = '''
WITH t1 AS (
SELECT
    user_id,
    event_id,
    video_id, 
    video_owner_id
FROM video_playlist_events
WHERE dt >= yesterday() - 14
AND event_type = 'add_video'
and playlist_owner_id < 0
and intHash64(user_id) % $shards_num = $shard
GROUP BY user_id, event_id, video_id, video_owner_id
),

t2 AS (
SELECT 
    user_id, 
    attachments.type, 
    attachments.owner_id, 
    attachments.item_id
FROM
    (SELECT 
        user_id,
        event_id,
        attachments.type, 
        attachments.owner_id, 
        attachments.item_id
    FROM posting 
    WHERE dt = yesterday()
    AND event_type LIKE 'post_%'
    AND owner_id < 0
    AND has(attachments.type, 'video')
    GROUP BY user_id, event_id, attachments.type, attachments.owner_id, attachments.item_id)
ARRAY JOIN attachments.type, attachments.owner_id, attachments.item_id
WHERE attachments.type = 'video'
and intHash64(user_id) % $shards_num = $shard
)

SELECT
    user_id
FROM t2
INNER JOIN t1 ON t1.video_id = t2.attachments.item_id AND t1.video_owner_id = t2.attachments.owner_id
'''

In [239]:
select100 = Select(client, sql_top, granularity=1)
select100.run()

{'operation_id': 'c6964450-25cb-4b57-a4c2-ecf9fb479580',
 'agg_operation_id': '899157db-ef29-4231-ab13-88c0ded63f2b',
 'replica_num': 1,
 'host': 'ch814601',
 'table_name': 'vtikhonov_2024_06_28_19_36_00_394174',
 'queue_position': 0,
 'pipeline_id': '899157db-ef29-4231-ab13-88c0ded63f2b',
 'check_version': False}

In [240]:
get_df_status(select100)

running agg 0/1
02_long
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
  agg   |  18:26:06  |  18:26:06  |  21600  | 226 |  33  |    o.belov    |be5ee9a6-7705-4804-9881-faad3d8efb0d|
 select |  19:36:26  |  19:36:27  |  21600  |  1  |  0   |  o.kulalaeva  |47999cab-2968-4161-81ed-105c6d14d41f|

03_normal
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  00:17:41  |     -      |   900   |  0  |  0   |   v.repnikov  |c13398d3-455a-4f7f-b107-97c313507e26|
 select |  00:17:41  |     -      |   900   |  0  |  0   |   v.repnikov  |2f914087-6fdf-4e52-9605-e61d9ebb8bf5|
 select |  00:17:41  |     -      |   900   |  0  |  0   |   v.repnik

In [241]:
select100.get_df(query='SELECT count() FROM $table')

,count()
0,208672


In [166]:
### Видео из сообщества (загрузил видео) если видео, а не клип

In [226]:
sql = '''
WITH t1 AS (
SELECT
    user_id,
    event_id,
    video_id, 
    video_owner_id
FROM video_publish
WHERE dt >= yesterday() - 14
AND event_type = 'publish_video'
and intHash64(user_id) % $shards_num = $shard
GROUP BY user_id, event_id, video_id, video_owner_id
),

t2 AS (
SELECT 
    user_id, 
    attachments.type, 
    attachments.owner_id, 
    attachments.item_id
FROM
    (SELECT 
        user_id,
        event_id,
        attachments.type, 
        attachments.owner_id, 
        attachments.item_id
    FROM posting 
    WHERE dt = yesterday()
    AND event_type LIKE 'post_%'
    AND owner_id < 0
    AND has(attachments.type, 'video')
    GROUP BY user_id, event_id, attachments.type, attachments.owner_id, attachments.item_id)
ARRAY JOIN attachments.type, attachments.owner_id, attachments.item_id
WHERE attachments.type = 'video'
and intHash64(user_id) % $shards_num = $shard
)

SELECT
    user_id
FROM t2
INNER JOIN t1 ON t1.video_id = t2.attachments.item_id AND t1.video_owner_id = t2.attachments.owner_id
'''

In [227]:
select = Select(client, sql, granularity=1)
select.run()

{'operation_id': 'b825c9ed-071f-4cc9-8447-5ec9359781ab',
 'agg_operation_id': '8127fa3e-ed68-48f4-9c89-7e3c2f783151',
 'replica_num': 0,
 'host': 'ch814600',
 'table_name': 'vtikhonov_2024_06_28_19_30_05_273964',
 'queue_position': 0,
 'pipeline_id': '8127fa3e-ed68-48f4-9c89-7e3c2f783151',
 'check_version': False}

In [231]:
get_df_status(select)

running agg 0/1
02_long
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
  agg   |  18:26:06  |  18:26:06  |  21600  | 226 |  28  |    o.belov    |be5ee9a6-7705-4804-9881-faad3d8efb0d|
 select |  19:04:40  |  19:04:41  |  21600  |  1  |  0   | e.kozhevnikov |ab815639-1291-4814-ac4a-8200961b3037|

03_normal
  Type  | created_at | started_at | timeout | all | done |    username   |            operation_id            |
---------------------------------------------------------------------------------------------------------------
 select |  00:17:41  |     -      |   900   |  0  |  0   |   v.repnikov  |c13398d3-455a-4f7f-b107-97c313507e26|
 select |  00:17:41  |     -      |   900   |  0  |  0   |   v.repnikov  |2f914087-6fdf-4e52-9605-e61d9ebb8bf5|
 select |  00:17:41  |     -      |   900   |  0  |  0   |   v.repnik

In [232]:
select.get_df(wait=True, query='SELECT count() FROM $table')

,count()
0,136825
